# Resumen

Este proyecto se desarrollo totalmente en los núcleos de kaggle. Se entrenan varios modelos y cada uno demanda varias horas de entrenamiento.<br>
Para poder hacer cambios y pruebas se implementó un sistema que activa o desactiva los entrenamientos y evaluaciones de cada modelo.<br>
XGBoost es el que más tiempo toma quedando en el limite de las 9h.<br><br>
Cada entrenamiento se hace independiente y posteriormente se juntan todos mendiante un staking
Los resultados se acumulan en un Dataset de kaggle https://www.kaggle.com/nesslc/sales-cv-for-s para leer desde ahi los archivos necesarios y finalmente entrenar un metamodel para hacer la predicción final.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn import model_selection


import math

from itertools import product
import ipywidgets as widgets
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm

import gc




In [ ]:
# Este es el vector que activa las partes en el siguiente orden:


#LastMonth,Baseline1,Baseline2,    XGBoost, LigthGB, SVM, MLP,    Reed from DS, Meta-model

partes_activas=[False,False,False,   False,False,False,False,        True,True]

# Cargar datos

In [ ]:
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
df_names = ['sales_train', 'shops','items', 'item_categories','test']

kaggle_data_tabs = widgets.Tab()
# Add Output widgets for each pandas DF as tabs' children
kaggle_data_tabs.children = list([widgets.Output() for df_name in df_names])

for index in range(0, len(df_names)):
    # Rename tab bar titles to df names
    kaggle_data_tabs.set_title(index, df_names[index])
    
    # Display corresponding table output for this tab name
    with kaggle_data_tabs.children[index]:
        display(eval(df_names[index]))

display(kaggle_data_tabs)

In [ ]:
print(len(sales_train['shop_id'].unique()),len(sales_train['item_id'].unique()))
print(len(sample_submission))

# Lo más básico de lo básico
La primera idea es ver que tan bien sale el score si usamos los datos del mes anterior y pasarlos tal cual como los datos de prediccion. 

In [ ]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

if partes_activas[0]:
    #Obtener las ventas totales de cada tienda-articulo 
    last = sales_train[sales_train['date_block_num']==33]
    grupos = last.groupby(['shop_id','item_id','date_block_num'])

    

    base0 = grupos.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
    base0 = base0.rename(columns = {'item_cnt_day' : 'item_cnt_month'})
    base0['item_cnt_month'] = base0['item_cnt_month'].clip(0,20)#Esto lo sugiere el organizador
    base0

In [ ]:
# Unir los valores obtenidos en el orden solicitado de ID para la submission final
if partes_activas[0]:
    submission_base0 = test.merge(base0,how='left',on=['shop_id','item_id']).fillna(0)
    submission_base0 = submission_base0.drop(columns=index_cols+['item_price'])
    submission_base0.to_csv('submision_base0.csv',index=False)
    submission_base0

# Un Modelo Basico
Ahora se entrenará un modelo base utilizando un modelo XGB  <br/>
Primero tengo que formar un set de entrenamiento

In [ ]:
#Se puede hacer una malla de cada mes con los pares de tienda-articulo
grid = []
for mes in sales_train['date_block_num'].unique():
    shops_mes = sales_train[sales_train['date_block_num']==mes]['shop_id'].unique()
    items_mes = sales_train[sales_train['date_block_num']==mes]['item_id'].unique()
    grid.append(np.array(list(product(*[shops_mes, items_mes, [mes]])),dtype='int32'))


full_grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# ahora unimos la información que tenemos

grupos = sales_train.groupby(index_cols)

sales_agg = grupos.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
sales_agg = sales_agg.rename(columns = {'item_cnt_day' : 'item_cnt_month'})
sales_agg['item_cnt_month'] = sales_agg['item_cnt_month'].clip(0,20)#Esto lo sugiere el organizador

train_set = full_grid.merge(sales_agg,how='left',on=index_cols).fillna(0)
# unimos la información de categoria de item
train_set = train_set.merge(items.drop(columns=['item_name']),how='left',on=['item_id']).astype({"item_cnt_month": int})
train_set

In [ ]:
features = ['shop_id','item_id','date_block_num','item_category_id']
X_train = train_set[features]#.set_index('shop_id')
y_train = train_set['item_cnt_month']
X_train

In [ ]:
# Make test_dataset pandas data frame, add category id and date block num, then convert back to numpy array and predict
merged_test = pd.merge(test, items,how='left', on = ['item_id'])[['shop_id','item_id','item_category_id']]
merged_test['date_block_num'] = 34
#merged_test = merged_test.set_index('shop_id')
merged_test

In [ ]:
if partes_activas[1]:
    model_base = xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1, eta = 0.3, n_estimators = 1000, seed = 1)
    model_base.fit(X_train, y_train, eval_metric='rmse')

    preds = model_base.predict(merged_test.values)

    preds_df = pd.DataFrame(preds,columns=['item_cnt_month'])
    preds_df['ID'] = preds_df.index
    preds_df = preds_df.set_index('ID')
    preds_df.to_csv('submision_base.csv')
    display(preds_df)

Esta versión basica da al rededor de 3.3 en el score lo cual es muuuy malo

# un poco de feature engenering
intentaremos hacer un poco de feature engenering y ajustar modelos mas trabajados. Las dos primeras cosas que se ocurren es agregar las ventas del mes o meses anteriores por cada tienda-articulo y la otra las ventas de ese articulo el mismo mes del año anterior.

In [ ]:
# todas las features que agreguemos se tienen que hacer en train y test

train_test = pd.concat([train_set, merged_test], axis = 0)



# Agregarems las ventas anteriores de cada mes durante todo un año para cada tienda-item 
meses_anteriores  = [1,2,3,12] 

for mes in tqdm(meses_anteriores):
    train_test_aux = train_test.copy()
    nombre_feature = 'prev_shopitem_sales_' + str(mes)
    train_test_aux.loc[:,'date_block_num'] += mes
    
    train_test_aux.rename(columns={'item_cnt_month': nombre_feature}, inplace=True)
    
    
    train_test = train_test.merge(train_test_aux[['shop_id', 'item_id', 'date_block_num', nombre_feature]], 
                                  on = ['shop_id', 'item_id', 'date_block_num'], how = 'left').fillna(0)

    

    
groups = train_test.groupby(by = ['item_id', 'date_block_num'])
for mes in tqdm(meses_anteriores):
    nombre_feature = 'prev_item_sales_' + str(mes)
    result = groups.agg({'item_cnt_month':'mean'})
    result = result.reset_index()
    result.loc[:, 'date_block_num'] += mes
    result.rename(columns={'item_cnt_month': nombre_feature}, inplace=True)
    train_test = train_test.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
    train_test[nombre_feature] = train_test[nombre_feature].fillna(0)
    
    


    
def past_information(df, merging_cols, new_col, aggregation):
    temp = []
    for i in tqdm(range(1,35)):
        block = df.query(f'date_block_num < {i}').groupby(merging_cols).agg(aggregation).reset_index()
        block.columns = merging_cols + [new_col]
        block['date_block_num'] = i
        block = block[block[new_col]>0]
        temp.append(block)
    temp = pd.concat(temp)
    df = pd.merge(df, temp, on=['date_block_num']+merging_cols, how='left')
    return df

#average item price in latest block item was sold
train_test = past_information(train_test, ['item_id'],'last_item_price',{'item_price':'last'})
           
#total units of item sold at individual shop
train_test = past_information(train_test, ['shop_id','item_id'],'item_cnt_sum_alltime',{'item_cnt_month':'sum'})

#total units of item sold at all shops
train_test = past_information(train_test, ['item_id'],'item_cnt_sum_alltime_allshops',{'item_cnt_month':'sum'})

train_test = past_information(train_test,)


## Mean encoding

In [ ]:
def mean_encode(train_test,to_encode):

    
    kf = model_selection.KFold(5, shuffle=False)
    feature_name = to_encode+'_enc'
    train_test[feature_name] = np.nan
    train_test[feature_name].fillna(0.3343, inplace=True)
    #all_data = train_test[train_test['date_block_num']<34]
    for tr_ind, val_ind in kf.split(train_test):
        X_tr, X_val = train_test.iloc[tr_ind], train_test.iloc[val_ind]
        train_test.loc[train_test.index[val_ind], feature_name] = X_val[to_encode].map(X_tr.groupby(to_encode).item_cnt_month.mean())
        #train_test[feature_name].fillna(0.3343, inplace=True)
    return train_test
    
    



In [ ]:
features_to_encode =['shop_id','item_id','item_category_id']

for element in tqdm(features_to_encode):
    train_test = mean_encode(train_test,element)

In [ ]:
train_test =train_test.fillna(0)
train_test

In [ ]:
if partes_activas[2]:
    msk = train_test['date_block_num']==34
    X_train = train_test[~msk]
    y_train = X_train['item_cnt_month']
    X_train = X_train.drop(columns=['item_cnt_month','item_price'])
    
    X_test = train_test[msk].drop(columns=['item_cnt_month','item_price'])

In [ ]:
if partes_activas[2]:
    model_base = xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1, eta = 0.3, n_estimators = 1000, seed = 1)
    model_base.fit(X_train, y_train, eval_metric='rmse')

    preds = model_base.predict(X_test.values)

    preds_df = pd.DataFrame(preds,columns=['item_cnt_month'])
    preds_df['ID'] = preds_df.index
    preds_df = preds_df.set_index('ID')
    preds_df.to_csv('submision1.csv')
    display(preds_df)

# Dividir mejor el train para hacer stacking

In [ ]:
if True:
    msk = train_test['date_block_num']<33
    X_train = train_test[msk]
    y_train = X_train['item_cnt_month']
    
    X_train = X_train.drop(columns=['item_cnt_month','item_price'])
    
    
    msk = train_test['date_block_num'] == 33
    X_val = train_test[msk]
    y_val = X_val['item_cnt_month'].astype('int8')
    
    X_val = X_val.drop(columns=['item_cnt_month','item_price'])
    
    msk = train_test['date_block_num'] == 34
    X_test = train_test[msk]
    X_test = X_test.drop(columns=['item_cnt_month','item_price'])
    


In [ ]:
del train_test
gc.collect()

# XGBoost

In [ ]:
if partes_activas[3]:
    XGBR_model = xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1, eta = 0.3, n_estimators = 1000, seed = 1)
    XGBR_model.fit(X_train, y_train, eval_metric='rmse')

    XGBR_test_preds = XGBR_model.predict(X_test.values)
    XGBR_val_preds = XGBR_model.predict(X_val.values)

    XGBpreds_df = pd.DataFrame(XGBR_test_preds,columns=['item_cnt_month'])
    XGBpreds_df['ID'] = XGBpreds_df.index
    XGBpreds_df = XGBpreds_df.set_index('ID')
    XGBpreds_df['item_cnt_month'] = XGBpreds_df['item_cnt_month'].clip(0,20)
    XGBpreds_df.to_csv('submisionXBG.csv')
    
    XGB_val_preds_df = pd.DataFrame(XGBR_val_preds,columns=['item_cnt_month'])
    XGB_val_preds_df['ID'] = XGB_val_preds_df.index
    XGB_val_preds_df = XGB_val_preds_df.set_index('ID')
    XGB_val_preds_df['item_cnt_month'] =  XGB_val_preds_df['item_cnt_month'].clip(0,20) 
    XGB_val_preds_df.to_csv('XBG_val_preds.csv')
    
    
    display(XGBpreds_df)
    
    del XGBR_model
    gc.collect()

en los nucleos de kaggle se tardó 3h en entrenar con 4 meses de features y mas de 6h con un año y elo vale la pena usar los 12 meses

# LightGBM

In [ ]:
def build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features):
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    model = lgb.train(params=params, train_set=lgb_train, valid_sets=(lgb_train, lgb_val), verbose_eval=50,
                     categorical_feature=cat_features)
    return model

In [ ]:
if partes_activas[4]:
    #skip this cell if directly loading saved model 
    params = {
        'objective': 'rmse',
        'metric': 'rmse',
        'num_leaves': 1023,
        'min_data_in_leaf':10,
        'feature_fraction':0.7,
        'learning_rate': 0.01,
        'num_rounds': 1000,
        'early_stopping_rounds': 30,
        'seed': 1
    }
    #designating the categorical features which should be focused on
    cat_features = ['date_block_num','shop_id','item_id']

    lgb_model = build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features)
    
    LGBM_test_preds = lgb_model.predict(X_test).clip(0,20)
    LGBM_val_preds = lgb_model.predict(X_val).clip(0,20)
    

In [ ]:
if partes_activas[4]:
    LGBpreds_df = pd.DataFrame(LGBM_test_preds,columns=['item_cnt_month'])
    LGBpreds_df['ID'] = LGBpreds_df.index
    LGBpreds_df = LGBpreds_df.set_index('ID')
    LGBpreds_df['item_cnt_month'] = LGBpreds_df['item_cnt_month'].clip(0,20)
    LGBpreds_df.to_csv('submisionLGBM.csv')
    
    LGB_val_preds_df = pd.DataFrame(LGBM_val_preds,columns=['item_cnt_month'])
    LGB_val_preds_df['ID'] = LGB_val_preds_df.index
    LGB_val_preds_df = LGB_val_preds_df.set_index('ID')
    LGB_val_preds_df['item_cnt_month'] =  LGB_val_preds_df['item_cnt_month'].clip(0,20) 
    LGB_val_preds_df.to_csv('LGBM_val_preds.csv')

# SVM 
No pude hacer correr un entrenamiento en el núcleo de kaggle

In [ ]:

if partes_activas[5]:
    svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
    svr_lin = SVR(kernel='linear', C=5, gamma='auto')
    svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
                   coef0=1)

    allSVM=[svr_lin]
    
    
    
    for i,VM in enumerate(allSVM):

        VM.fit(X_train, y_train)
        nombre='SVM_val_'+str(i)+'.csv'
        SVM_val_preds=VM.predict(X_val)

        SVM_val_preds_df = pd.DataFrame(SVM_val_preds,columns=['item_cnt_month'])
        SVM_val_preds_df['ID'] = SVM_val_preds_df.index
        SVM_val_preds_df = SVM_val_preds_df.set_index('ID')
        SVM_val_preds_df['item_cnt_month'] =  SVM_val_preds_df['item_cnt_month'].clip(0,20) 
        SVM_val_preds_df.to_csv(nombre)


        nombre='SVM_test_'+str(i)+'.csv'
        SVM_test_preds=VM.predict(X_test)

        SVM_test_preds_df = pd.DataFrame(SVM_test_preds,columns=['item_cnt_month'])
        SVM_test_preds_df['ID'] = SVM_test_preds_df.index
        SVM_test_preds_df = SVM_test_preds_df.set_index('ID')
        SVM_test_preds_df['item_cnt_month'] =  SVM_test_preds_df['item_cnt_month'].clip(0,20) 
        SVM_test_preds_df.to_csv(nombre)
    

# Ahora intento algo de Deeplearning

In [ ]:
class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        
        X = self.list_IDs[index]
        
        y = self.labels[index]
        

        return X, y



In [ ]:
y_trainT = torch.tensor(y_train.values)
del y_train
X_trainT = torch.tensor(X_train.values)
del X_train
y_valT = torch.tensor(y_val.values)
X_valT = torch.tensor(X_val.values)
X_testT = torch.tensor(X_test.values)
numero_features = X_trainT.size()[1]


In [ ]:
gc.collect()

In [ ]:
def make_DL(DSE,labels):
    DS = Dataset(DSE, labels)
    generator = DataLoader(DS,batch_size=102400, shuffle=False)
    return DS , generator 
    



In [ ]:
X_trainT, train_dl = make_DL(X_trainT,y_trainT)
X_valT, val_dl = make_DL(X_valT, y_valT)


In [ ]:
print('\nFirst iteration of data set: ', next(iter(train_dl)), '\n')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [ ]:
#nn package to create our linear model
# each Linear module has a weight and bias

torch.manual_seed(11)
learning_rate = 1e-3
lambda_l2 = 1e-5

F = numero_features
H1 = 512
H2 = 1024
H3 = 1024
H4 = 64
H5 = 16
S = 1



A1MLP = nn.Sequential(
    nn.Linear(F, H2),
    nn.ReLU(),
    nn.Linear(H2, H4),
    nn.ReLU(),
    nn.Linear(H4, S),
    
)

A2MLP = nn.Sequential(
    nn.Linear(F, H1),
    nn.ReLU(),
    nn.Linear(H1, H2),
    nn.ReLU(),
    nn.Linear(H2, H3),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(H3, H4),
    nn.ReLU(),
    nn.Linear(H4, H5),
    nn.ReLU(),
    nn.Linear(H5, S),
    
)

MLP_arch =[A1MLP,A2MLP]



In [ ]:
if partes_activas[6]:
    
    for i,MLP in enumerate(MLP_arch):
    
        MLP.to(device)


        criterion = torch.nn.MSELoss()

        # we use the optim package to apply
        # ADAM for our parameter updates
        optimizer = torch.optim.Adam(MLP.parameters(), lr=learning_rate, weight_decay=lambda_l2) 

        epochs = 3+i

        for epoch in range(epochs):
        # Training
            MLP.train()
            epoch_loss=0
            for batch in tqdm(train_dl):
                # Feed forward to get the logits
                Xmin,ymin = batch
                Xmin = Xmin.float()
                ymin=ymin.float()
                y_pred = torch.squeeze(MLP(Xmin.to(device)))

                # Compute the loss 
                loss = criterion(y_pred.to(device), ymin.to(device))
                epoch_loss = loss.item() + epoch_loss
                #print(epoch_loss)
                #acc = (y == predicted).sum().float() / len(y)

                #display.clear_output(wait=True)

                # zero the gradients before running
                # the backward pass.
                optimizer.zero_grad()

                # Backward pass to compute the gradient
                # of loss w.r.t our learnable params. 
                loss.backward()

                # Update params
                optimizer.step()
            epoch_loss = epoch_loss/len(train_dl)


            #Validation Step
            MLP.eval()
            val_loss=0
            y_pred_list=[]
            for batch in tqdm(val_dl):
                Xmin,ymin = batch
                Xmin = Xmin.float()
                ymin=ymin.float()

                y_pred = torch.squeeze(MLP(Xmin.to(device)))
                y_pred_list = y_pred_list + y_pred.tolist()

                lossV = criterion(y_pred.to(device), ymin.to(device))
                val_loss = lossV.item() + val_loss
            val_loss = val_loss/len(val_dl)

            val_score = mean_squared_error(y_val,np.array(y_pred_list),squared=False)

            print("[EPOCH]: %i, [LOSS_TRAIN]: %.6f, [LOSS_VAL]: %.6f, [VAL_SCORE]: %.6f" % (epoch, epoch_loss, val_loss, val_score))

        with torch.no_grad():
            MLPpreds = MLP(X_testT.float().to(device))

        MLPpreds_df = pd.DataFrame(MLPpreds.cpu().numpy(),columns=['item_cnt_month'])
        MLPpreds_df['ID'] = MLPpreds_df.index
        MLPpreds_df = MLPpreds_df.set_index('ID')
        MLPpreds_df['item_cnt_month'] = MLPpreds_df['item_cnt_month'].clip(0,20)
        MLPpreds_df.to_csv('submisionMLP'+str(i)+'.csv')


        MLP_val_preds_df = pd.DataFrame(y_pred_list,columns=['item_cnt_month'])
        MLP_val_preds_df['ID'] = MLP_val_preds_df.index
        MLP_val_preds_df = MLP_val_preds_df.set_index('ID')
        MLP_val_preds_df['item_cnt_month']=MLP_val_preds_df['item_cnt_month'].clip(0,20)
        MLP_val_preds_df.to_csv('MLP_val_preds'+str(i)+'.csv')

        display(MLPpreds_df)

    
    


# Stacking con los modelos

In [ ]:
if partes_activas[7]:
    
    
    MLP_val_preds_df1  = pd.read_csv('../input/sales-cv-for-s/MLP_val_preds1.csv')
    MLP_val_preds_df1['item_cnt_month']=MLP_val_preds_df1['item_cnt_month'].clip(0,20)
    
    MLPpreds_df1 =  pd.read_csv('../input/sales-cv-for-s/submisionMLP1.csv')
    
    
    
    MLP_val_preds_df2  = pd.read_csv('../input/sales-cv-for-s/MLP_val_preds0.csv')
    MLP_val_preds_df2['item_cnt_month']=MLP_val_preds_df2['item_cnt_month'].clip(0,20)
    
    MLPpreds_df2 =  pd.read_csv('../input/sales-cv-for-s/submisionMLP0.csv')
    
    XGB_val_preds_df = pd.read_csv('../input/sales-cv-for-s/XBG_val_preds.csv')
    XGB_val_preds_df['item_cnt_month'] =  XGB_val_preds_df['item_cnt_month'].clip(0,20) 
    
    XGBpreds_df = pd.read_csv('../input/sales-cv-for-s/submisionXBG.csv')
    
    LGB_val_preds_df = pd.read_csv('../input/sales-cv-for-s/LGBM_val_preds.csv')
    LGB_val_preds_df['item_cnt_month'] =  LGB_val_preds_df['item_cnt_month'].clip(0,20) 
    
    LGBpreds_df = pd.read_csv('../input/sales-cv-for-s/submisionLGBM.csv')

In [ ]:
y_val.reset_index().head(30)

In [ ]:
if partes_activas[8]:

    X_train_level2= pd.concat([MLP_val_preds_df1['item_cnt_month'],XGB_val_preds_df['item_cnt_month'],LGB_val_preds_df['item_cnt_month'], MLP_val_preds_df2['item_cnt_month']],axis=1)
    X_test_level2 = pd.concat([MLPpreds_df1['item_cnt_month'],XGBpreds_df['item_cnt_month'],LGBpreds_df['item_cnt_month'],MLPpreds_df2['item_cnt_month']],axis=1)

    from sklearn.linear_model import LinearRegression as lr
    from sklearn.ensemble import RandomForestRegressor


    #meta_model = SVR(kernel='linear', C=20, gamma='auto',verbose=True)
    meta_model = lr()
    meta_model.fit(X_train_level2, y_val)

    predsMeta=meta_model.predict(X_test_level2)



    Meta_preds_df = pd.DataFrame(predsMeta,columns=['item_cnt_month'])
    Meta_preds_df['ID'] = Meta_preds_df.index
    Meta_preds_df = Meta_preds_df.set_index('ID')
    Meta_preds_df['item_cnt_month'] = Meta_preds_df['item_cnt_month'].clip(0,20)
    Meta_preds_df.to_csv('submisionMeta.csv')

In [ ]:
Meta_preds_df.head(30)